In [1]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: default.

In [ ]:
from aiida import orm
from aiida_openbis.utils import bisutils

In [ ]:
def find_first_workchain(node):
    """Find the first workchain in the provenance that created the structure node."""
    lastcalling = None
    if isinstance(node, orm.StructureData):
        previous_node = node.creator
    else:
        previous_node = node
    while previous_node is not None:
        lastcalling = previous_node
        previous_node = lastcalling.caller
    if lastcalling is not None:
        return lastcalling.label, lastcalling.pk, lastcalling.description
    return None, None, ""

In [36]:
# Connect to openBIS
session = bisutils.log_in(bisurl="openbis", bisuser="admin", bispasswd="changeit")

In [30]:
pk = 62
workchain = load_node(pk)
structure = workchain.inputs.structure

all_structures = [structure]
creator_found = True

while creator_found:
    creator = structure.creator
    if creator is not None:
        structure = creator.inputs.structure
        all_structures.append(structure)
    else:
        creator_found = False

[<StructureData: uuid: 5bbc1f32-3000-4c57-83e1-135cc4701410 (pk: 34)>, <StructureData: uuid: 83622d39-975c-49a2-972b-e86f2544fcb3 (pk: 51)>]


In [ ]:
geoopt_workchain_details = find_first_workchain(load_node(34))
geoopt_workchain_pk = geoopt_workchain_details[1]

geoopt_workchain = load_node(geoopt_workchain_pk)

In [48]:
pk = 62
workchain = load_node(pk)

simulated_stm_uuid = workchain.uuid
structure = workchain.inputs.structure
dft_params = dict(workchain.inputs.dft_params)
stm_params = dict(workchain.inputs.stm_params)

collection_code = "/CARBON_NANOMATERIALS/TRIANGULENE_SPIN_CHAINS/TRIANGULENE_SPIN_CHAINS_EXP_3"
new_spm = session.new_sample(collection=collection_code, type='STM_SIMULATION')
new_spm.props['$name'] = "Simulated STM"
new_spm.props['description'] = "A nice simulated STM"
new_spm.props['e-min'] = stm_params['--energy_range'][0]
new_spm.props['e-max'] = stm_params['--energy_range'][1]
new_spm.props['de'] = stm_params['--energy_range'][2]
new_spm.save()


sample successfully created.


In [65]:
workchain.inputs.structure

<StructureData: uuid: 5bbc1f32-3000-4c57-83e1-135cc4701410 (pk: 34)>

In [82]:
find_first_workchain(load_node(34))

('CP2K_GeoOpt', 81, 'A:Pd1_mol1_S_redone')

In [94]:
x = load_node(81)
structure = x.inputs.structure
structure_ase = structure.get_ase()
structure_ase.positions
structure_ase.symbols
structure_ase.pbc
structure_ase.cell
structure_ase.get_cell()

Cell([27.799952505728, 36.113197640942, 55.799336300735])

In [75]:
geo_opt_workchain.get_ase()

Atoms(symbols='C16H14Ga552OPd576', pbc=False, cell=[27.799952505728, 36.113197640942, 55.799336300735], masses=..., tags=...)

In [51]:
dict(geo_opt_workchain.inputs.parameters)

{'GLOBAL': {'RUN_TYPE': 'GEO_OPT',
  'WALLTIME': 85800,
  'PRINT_LEVEL': 'MEDIUM',
  'EXTENDED_FFT_LENGTHS': ''},
 'MOTION': {'PRINT': {'RESTART_HISTORY': {'_': 'OFF'}},
  'GEO_OPT': {'BFGS': {'TRUST_RADIUS': '[bohr] 0.1'},
   'MAX_DR': 0.003,
   'RMS_DR': 0.001,
   'MAX_ITER': 1000,
   'MAX_FORCE': 0.0001,
   'OPTIMIZER': 'BFGS',
   'RMS_FORCE': 0.0001},
  'CONSTRAINT': {'FIXED_ATOMS': {'LIST': '584..1159 '}}},
 'FORCE_EVAL': {'DFT': {'QS': {'METHOD': 'GPW',
    'EPS_DEFAULT': 1e-14,
    'EXTRAPOLATION': 'ASPC',
    'EXTRAPOLATION_ORDER': 3},
   'XC': {'VDW_POTENTIAL': {'PAIR_POTENTIAL': {'TYPE': 'DFTD3',
      'R_CUTOFF': 15,
      'CALCULATE_C9_TERM': '.TRUE.',
      'PARAMETER_FILE_NAME': 'dftd3.dat',
      'REFERENCE_FUNCTIONAL': 'PBE'},
     'DISPERSION_FUNCTIONAL': 'PAIR_POTENTIAL'},
    'XC_FUNCTIONAL': {'_': 'PBE'}},
   'SCF': {'OT': {'MINIMIZER': 'CG', 'PRECONDITIONER': 'FULL_SINGLE_INVERSE'},
    'PRINT': {'RESTART': {'EACH': {'QS_SCF': 0, 'GEO_OPT': 1},
      'ADD_LAST': 'N